## Processamento de Dados

In [37]:
import requests

url = "https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv"
output_path = "20250401_atividade_economica.csv"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    with open(output_path, 'wb') as f:
        f.write(response.content)
    print(f"Arquivo salvo como: {output_path}")
else:
    print(f"Erro ao baixar: {response.status_code}")


Arquivo salvo como: 20250401_atividade_economica.csv


In [3]:
import pandas as pd

df = pd.read_csv("20250401_atividade_economica.csv", sep=";", encoding="utf-8")
print(df.columns)
print(f"Registros iniciais: {len(df)}")

Index(['ID_ATIV_ECON_ESTABELECIMENTO', 'CNAE_PRINCIPAL',
       'DESCRICAO_CNAE_PRINCIPAL', 'CNAE', 'DATA_INICIO_ATIVIDADE',
       'NATUREZA_JURIDICA', 'PORTE_EMPRESA', 'AREA_UTILIZADA', 'IND_SIMPLES',
       'IND_MEI', 'IND_POSSUI_ALVARA', 'TIPO_UNIDADE', 'FORMA_ATUACAO',
       'DESC_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'COMPLEMENTO',
       'NOME_BAIRRO', 'NOME', 'NOME_FANTASIA', 'CNPJ', 'GEOMETRIA'],
      dtype='object')
Registros iniciais: 534880


In [6]:
df['DESCRICAO_CNAE_PRINCIPAL'] = df['DESCRICAO_CNAE_PRINCIPAL'].fillna("").str.upper()

keywords = ["BAR", "RESTAURANTE", "BARES", "RESTAURANTES"]

filter = df['DESCRICAO_CNAE_PRINCIPAL'].apply(lambda x: any(k in x for k in keywords))
df_filter = df[filter].copy()

print(f"Registros restantes: {len(df_filter)}")


Registros restantes: 13801


In [23]:
important_data = [
    'DATA_INICIO_ATIVIDADE', 
    'IND_POSSUI_ALVARA', 
    'NOME',
    'DESC_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'NOME_BAIRRO'
]

df_final = df_filter[important_data].copy()

df_final['ENDERECO'] = (
    df_final['DESC_LOGRADOURO'].fillna('') + ' ' +
    df_final['NOME_LOGRADOURO'].fillna('') + ', ' +
    df_final['NUMERO_IMOVEL'].fillna('S/N').astype(str) + ' - ' +
    df_final['NOME_BAIRRO'].fillna('')
)

df_final = df_final[['NOME', 'DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO']]

print(df_final.head())

                                       NOME DATA_INICIO_ATIVIDADE  \
17                 APARECIDA MARIA DE SOUZA            01-07-1993   
19               LUCAS SILVA DE ARAUJO LTDA            15-10-1993   
29                      PAULO EMILIO COELHO            02-05-1994   
172          GARAPAO BAR E RESTAURANTE LTDA            24-11-1993   
196  ROD BITS COMERCIO E REPRESENTACAO LTDA            01-06-1994   

    IND_POSSUI_ALVARA                                           ENDERECO  
17                NÃO  RUA DESEMBARGADOR REIS ALVES, 90 - BAIRRO DAS ...  
19                NÃO                  RUA LUIZ PONGELUPE, 290 - CARDOSO  
29                NÃO                 AVE RESSACA, 118 - PADRE EUSTAQUIO  
172               SIM   RUA DESEMBARGADOR RIBEIRO DA LUZ, 135 - BARREIRO  
196               NÃO              AVE ELIAS ANTONIO ISSA, 288 - LETICIA  


## Geocodificação

In [24]:
%pip install geopy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="pbh-geocodificador")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

In [31]:
def convert_coordinates(address: str):
    try:
        location = geocode(address)
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        print(f"Erro ao geocodificar '{address}': {e}")
    return pd.Series([None, None])

In [ ]:
df_final[['LATITUDE', 'LONGITUDE']] = df_final['ENDERECO'].apply(convert_coordinates)
print(df_final.head())